In [2]:
import pandas as pd

transactions = pd.read_csv("transactions.csv", sep=',')
tr_mcc_codes = pd.read_csv("tr_mcc_codes.csv", sep=';')
gender_train = pd.read_csv("gender_train.csv", sep=',')
tr_types = pd.read_csv("tr_types.csv", sep=';')

(6849346, 6)

## Задание 1


1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

In [3]:
df = transactions['tr_type'].sample(1000)
df1 = pd.merge(df, tr_types, on='tr_type')
df2 = df1[df1['tr_description'].apply(lambda x: 'POS' in x or 'ATM' in x)]
print(len(df2) / len(df1))

0.388


(6849346, 6)

## Задание 2
1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости.

In [4]:
df = transactions['tr_type'].value_counts()
top_tr = df.index[0:10]
print(df.head(10))

1010    1589734
2010    1040713
1030     970994
1110     928154
7070     691896
2370     376521
7010     258088
7030     155200
1100     117870
7071      96146
Name: tr_type, dtype: int64


(6849346, 6)

## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите разницу для этих клиентов между суммой расходов и суммой приходов

In [5]:
df = transactions[transactions['amount'] > 0]
users = df.groupby('customer_id').sum()
client1 = users['amount'].idxmax()
print(client1)

df = transactions[transactions['amount'] > 0]
users = df.groupby('customer_id').sum()
client2 = users['amount'].idxmin()
print(client2)

df = transactions.groupby('customer_id').sum()
print(df.loc[client1]['amount'])
print(df.loc[client2]['amount'])

48281795
34419732
365148.43999981915
-4222747.95


(6849346, 6)

## Задание 4
1. Найдите среднее арифметическое и медиану по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по всем типам транзакций для клиентов из задания 3

In [6]:
df = transactions[transactions['tr_type'].apply(lambda x: x in top_tr[0:10])]
df1 = df.groupby('tr_type').mean()
df1.head(10)

(6849346, 6)

In [7]:
df.groupby('tr_type').median().head(10)

,customer_id,mcc_code,amount
tr_type,,,
1010,50000543.0,5499.0,-6733.26
1030,49913122.0,4814.0,-2245.92
1100,52912415.0,5999.0,-8983.66
1110,50774566.0,5499.0,-9365.47
2010,50081041.0,6011.0,-44918.32
2370,49004865.0,4829.0,-43795.36
7010,50095077.0,6011.0,93205.50
7030,52428903.5,6010.0,22459.16
7070,54888680.0,6010.0,14284.02


In [8]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')

(6846580, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том,
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [9]:
df = transactions.groupby(['gender'])
df_men = df.get_group(1)
df_women = df.get_group(0)
df_men_outcomes = df_men[df_men.amount < 0]
df_women_outcomes = df_women[df_women.amount < 0]
print(abs(df_women_outcomes.amount.sum() - df_men_outcomes.amount.sum()))
df_men_incomes = df_men[df_men.amount > 0]
df_women_incomes = df_women[df_women.amount > 0]
print(abs(df_women_incomes.amount.sum() - df_men_incomes.amount.sum()))

29317384780.10005
15367781650.390015


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 транзакций для мужчин и для женщин, наименьших среди всех транзакций по полученным значениям "max_income".
2. Выделите среди них те, которые встречаются одновременно и у мужчин, и у женщин

In [31]:
df_men=df_men[df_men.amount>0]
men_max_income=df_men.groupby('tr_type').amount.max()
men_max_income.sort_values(inplace=True)
men_max_income=men_max_income[0:10]
print(men_max_income)

tr_type
4051      381.81
4071      673.77
4500     2245.92
4110     3144.28
4210     4166.17
4010     4469.37
4100     5220.63
8146    22459.16
2331    38180.57
1410    44918.32
Name: amount, dtype: float64


In [30]:
df_women=df_women[df_women.amount>0]
women_max_income=df_women.groupby('tr_type').amount.max()
women_max_income.sort_values(inplace=True)
women_max_income=women_max_income[0:10]
print(women_max_income)

tr_type
4011       50.53
4021      592.70
4071     1010.66
4051     1122.96
4110     4469.37
4100     5041.86
4210     5246.91
2011     6737.75
8146    14823.04
1030    31128.39
Name: amount, dtype: float64


In [38]:
idx1 = men_max_income.index
idx2 = women_max_income.index
idx=idx1.intersection(idx2)
print(idx)

Int64Index([4051, 4071, 4110, 4210, 4100, 8146], dtype='int64', name='tr_type')


## Задание 7

1. Найдите суммы затрат по каждой категории для мужчин и для женщин.
2. Найдите топ 5 категорий с самыми большими относительными модулями разности для в тратах для разных полов.